# Modeling: Подготовка датасетов для моделей и создание самих моделей

## 1. Импорт библиотек

Импортируем стандартные библиотеки для анализа данных, а также пользовательские функции для визуализации и первичного осмотра данных.

In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

try:
    from utils.paths import join_path as pj
except ImportError:
    from scooter_sharing_analysis.utils.paths import join_path as pj

## 2. Загрузка очищенных данных

Загружаем файл rides_weather_data.csv нормализованный после очистки.

In [9]:
# Настраиваем словари с параметрами чтения CSV: путь, кодировка, дата-колонки
rides_options = {
    "filepath_or_buffer": pj("data", "rides_weather_data.csv"),
    "encoding": "utf-8",
    "parse_dates": ["start_date", "end_date", "day_timestamp", "hour_timestamp"],
}

rides_weather_data = pd.read_csv(**rides_options)

## 3. Создание факторов для моделей: временные, лаговые, погодные

## 3.1. Создание временных и лаговых факторов

Группируем данные по часовому ряду, получая спрос за каждый час (demand), добавляет новую колонку: 'hour_of_day'. Делим данные на тестовые и тренировочные с соотношение 1:4, добавляем лаговые колонки: 'lag_1h', 'lag_24h', 'mean_last_24h', 'mean_last_7d'

In [10]:
# Группировка данных и получение спроса
demand = rides_weather_data.groupby("hour_timestamp").agg({"id": "count"}).rename(columns={"id": "demand"}).reset_index()

# Добавление новых колонки на основе "hour_timestamp"
demand["hour_of_day"] = demand["hour_timestamp"].dt.hour

# Создание лаговых признаков
demand["lag_1h"] = demand["demand"].shift(1)
demand["lag_24h"] = demand["demand"].shift(24)
demand["mean_last_24h"] = demand["lag_1h"].rolling(24).mean()
demand["mean_last_7d"] = demand["lag_1h"].rolling(24 * 7).mean()

# Удаление строк с Nan значениями в лагах
demand = demand.dropna()

# Разбиение данных на тестувую и тренировочную части
X_demand_train, X_demand_test, Y_demand_train, Y_demand_test = train_test_split(demand, demand["demand"], test_size=0.2, shuffle=False)

# Вывод первой 1000 строк
X_demand_train.head(1000)


,hour_timestamp,demand,hour_of_day,lag_1h,lag_24h,mean_last_24h,mean_last_7d
168,2023-04-29 09:00:00,31,9,24.0,28.0,34.375000,31.898810
169,2023-04-29 10:00:00,28,10,31.0,24.0,34.500000,31.970238
170,2023-04-29 11:00:00,31,11,28.0,18.0,34.666667,31.892857
171,2023-04-29 12:00:00,44,12,31.0,21.0,35.208333,31.916667
172,2023-04-29 13:00:00,47,13,44.0,25.0,36.166667,31.928571
...,...,...,...,...,...,...,...
1163,2023-06-09 22:00:00,65,22,61.0,28.0,40.666667,41.190476
1164,2023-06-09 23:00:00,37,23,65.0,27.0,42.208333,41.136905
1165,2023-06-10 00:00:00,17,0,37.0,6.0,42.625000,41.184524
1166,2023-06-10 01:00:00,12,1,17.0,2.0,43.083333,41.119048


## 3.2. OHE кодирование категориальных признаков: start_location, promo

Создаем модель OHE для кодирования категориальных признаков "start_location", "promo", получаем имена будущих колонок для датафрейма с закодированными данными, на основе закодированных данных и имен колонок создаем новый датафрейм, который объединяем по оси 1 с начальным датафреймом предварительно удалив уже ненужные категориальные признаки "start_location", "promo"

In [11]:
# Создание OHE модели с заданными параметрами
encoder = OneHotEncoder(drop="first", sparse_output=False)

# Название колонок с категориальными признаками
column_names = ["start_location", "promo"]

# Кодирование категориальных признаков на основе модели
encoded_array = encoder.fit_transform(rides_weather_data[column_names])

# Получение названий колонок для будущего датафрейма
feat_names = encoder.get_feature_names_out(column_names)

# Создание датафрейма с закодированными признаками
encoded_data = pd.DataFrame(encoded_array, columns=feat_names)

# Объединение датафреймов по 1 оси
encoded_rides_weather_data = pd.concat([rides_weather_data.drop(columns=column_names, axis=1), encoded_data], axis=1)

# Вывод получившегося датафрейма
encoded_rides_weather_data

,id,start_date,end_date,day_of_week,start_district,end_location,end_district,distance,duration_minutes,total_price,...,start_location_чехова,start_location_чкалова,start_location_шевченко,start_location_школьная,start_location_шоссейная,start_location_энергетиков,start_location_энтузиастов,start_location_юбилейная,start_location_южная,promo_Нет
0,100340,2023-04-22 09:01:03,2023-04-22 09:24:45,суббота,юго-восточный,майская,октябрьский,4069.0,24.0,126.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,100341,2023-04-22 09:01:07,2023-04-22 09:29:32,суббота,заречный,спортивная,октябрьский,4515.0,28.0,142.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,100342,2023-04-22 09:04:27,2023-04-22 09:42:46,суббота,октябрьский,суворова,ленинский,4520.0,38.0,182.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,100343,2023-04-22 09:06:34,2023-04-22 09:38:23,суббота,центральный,рабочая,северо-западный,3881.0,32.0,158.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,100344,2023-04-22 09:09:27,2023-04-22 09:40:07,суббота,северо-западный,юбилейная,юго-восточный,4029.0,31.0,154.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93281,197707,2023-07-31 23:45:06,2023-08-01 00:06:55,понедельник,юго-восточный,первомайская,северо-западный,3302.0,22.0,140.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
93282,197708,2023-07-31 23:46:01,2023-08-01 00:25:35,понедельник,ленинский,садовая,заречный,4992.0,40.0,230.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
93283,197709,2023-07-31 23:48:04,2023-08-01 00:09:21,понедельник,ленинский,куйбышева,центральный,2781.0,21.0,135.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
93284,197710,2023-07-31 23:51:41,2023-08-01 00:38:04,понедельник,северо-западный,энергетиков,юго-восточный,6466.0,46.0,260.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
